# Demo vectorbt

Library website: https://github.com/polakowo/vectorbt

Example this file based on: https://vectorbt.dev/#example

In [1]:
import numpy as np
import pandas as pd
from datetime import datetime
import matplotlib.pyplot as plt

import vectorbt as vbt

In [2]:
# Prepare data
start = "2022-01-01 UTC" # Crypto is in UTC
end = "2025-01-01 UTC"
btc_price = vbt.YFData.download(symbols="BTC-USD",
                                start=start,
                                end=end).get("Close")
type(btc_price)

pandas.core.series.Series

In [3]:
btc_price.head()

Date
2022-01-01 00:00:00+00:00    47686.812500
2022-01-02 00:00:00+00:00    47345.218750
2022-01-03 00:00:00+00:00    46458.117188
2022-01-04 00:00:00+00:00    45897.574219
2022-01-05 00:00:00+00:00    43569.003906
Freq: D, Name: Close, dtype: float64

In [4]:
btc_price.tail()

Date
2024-12-27 00:00:00+00:00    94164.859375
2024-12-28 00:00:00+00:00    95163.929688
2024-12-29 00:00:00+00:00    93530.226562
2024-12-30 00:00:00+00:00    92643.210938
2024-12-31 00:00:00+00:00    93429.203125
Freq: D, Name: Close, dtype: float64

We are going to test a simple Dual Moving Average Crossover (DMAC) strategy. For this, we are going to use `MA` class for calculating moving averages and generating signals.

Our first test is rather simple: buy when the 10-day moving average crosses above the 20-day moving average, and sell when opposite.

In [5]:
fast_ma = vbt.MA.run(btc_price, 10, short_name='fast')
slow_ma = vbt.MA.run(btc_price, 20, short_name='slow')

In [6]:
type(fast_ma)

vectorbt.indicators.basic.MA

In [7]:
entries = fast_ma.ma_crossed_above(slow_ma)
entries

Date
2022-01-01 00:00:00+00:00    False
2022-01-02 00:00:00+00:00    False
2022-01-03 00:00:00+00:00    False
2022-01-04 00:00:00+00:00    False
2022-01-05 00:00:00+00:00    False
                             ...  
2024-12-27 00:00:00+00:00    False
2024-12-28 00:00:00+00:00    False
2024-12-29 00:00:00+00:00    False
2024-12-30 00:00:00+00:00    False
2024-12-31 00:00:00+00:00    False
Freq: D, Length: 1096, dtype: bool

In [8]:
exits = fast_ma.ma_crossed_below(slow_ma)
exits

Date
2022-01-01 00:00:00+00:00    False
2022-01-02 00:00:00+00:00    False
2022-01-03 00:00:00+00:00    False
2022-01-04 00:00:00+00:00    False
2022-01-05 00:00:00+00:00    False
                             ...  
2024-12-27 00:00:00+00:00    False
2024-12-28 00:00:00+00:00    False
2024-12-29 00:00:00+00:00    False
2024-12-30 00:00:00+00:00    False
2024-12-31 00:00:00+00:00    False
Freq: D, Length: 1096, dtype: bool

In [9]:
pf = vbt.Portfolio.from_signals(btc_price, entries, exits)
pf.total_return()

np.float64(0.3033659154521979)

In [10]:
# Multiple strategy instances: (10, 30) and (20, 30)
fast_ma = vbt.MA.run(btc_price, [10, 20], short_name='fast')
slow_ma = vbt.MA.run(btc_price, [30, 30], short_name='slow')

In [11]:
entries = fast_ma.ma_crossed_above(slow_ma)
entries

fast_window,10,20
slow_window,30,30
Date,,
2022-01-01 00:00:00+00:00,False,False
2022-01-02 00:00:00+00:00,False,False
2022-01-03 00:00:00+00:00,False,False
2022-01-04 00:00:00+00:00,False,False
2022-01-05 00:00:00+00:00,False,False
...,...,...
2024-12-27 00:00:00+00:00,False,False
2024-12-28 00:00:00+00:00,False,False


In [12]:
exits = fast_ma.ma_crossed_below(slow_ma)
exits

fast_window,10,20
slow_window,30,30
Date,,
2022-01-01 00:00:00+00:00,False,False
2022-01-02 00:00:00+00:00,False,False
2022-01-03 00:00:00+00:00,False,False
2022-01-04 00:00:00+00:00,False,False
2022-01-05 00:00:00+00:00,False,False
...,...,...
2024-12-27 00:00:00+00:00,False,False
2024-12-28 00:00:00+00:00,False,False


In [13]:
pf = vbt.Portfolio.from_signals(btc_price, entries, exits)
pf.total_return()

fast_window  slow_window
10           30             0.579169
20           30             0.579747
Name: total_return, dtype: float64

In [14]:
# Multiple strategy instances and instruments
eth_price = vbt.YFData.download('ETH-USD', start=start, end=end).get('Close')
comb_price = btc_price.vbt.concat(eth_price,
                                  keys=pd.Index(['BTC', 'ETH'], name='symbol'))
comb_price.vbt.drop_levels(-1, inplace=True)
comb_price

symbol,BTC,ETH
Date,,
2022-01-01 00:00:00+00:00,47686.812500,3769.697021
2022-01-02 00:00:00+00:00,47345.218750,3829.564941
2022-01-03 00:00:00+00:00,46458.117188,3761.380371
2022-01-04 00:00:00+00:00,45897.574219,3794.056641
2022-01-05 00:00:00+00:00,43569.003906,3550.386963
...,...,...
2024-12-27 00:00:00+00:00,94164.859375,3328.916992
2024-12-28 00:00:00+00:00,95163.929688,3397.902344
2024-12-29 00:00:00+00:00,93530.226562,3349.513428


In [15]:
fast_ma = vbt.MA.run(comb_price, [10, 20], short_name='fast')
slow_ma = vbt.MA.run(comb_price, [30, 30], short_name='slow')

In [16]:
entries = fast_ma.ma_crossed_above(slow_ma)
entries

fast_window                   10            20       
slow_window                   30            30       
symbol                       BTC    ETH    BTC    ETH
Date                                                 
2022-01-01 00:00:00+00:00  False  False  False  False
2022-01-02 00:00:00+00:00  False  False  False  False
2022-01-03 00:00:00+00:00  False  False  False  False
2022-01-04 00:00:00+00:00  False  False  False  False
2022-01-05 00:00:00+00:00  False  False  False  False
...                          ...    ...    ...    ...
2024-12-27 00:00:00+00:00  False  False  False  False
2024-12-28 00:00:00+00:00  False  False  False  False
2024-12-29 00:00:00+00:00  False  False  False  False
2024-12-30 00:00:00+00:00  False  False  False  False
2024-12-31 00:00:00+00:00  False  False  False  False

[1096 rows x 4 columns]

In [17]:
exits = fast_ma.ma_crossed_below(slow_ma)
exits

fast_window                   10            20       
slow_window                   30            30       
symbol                       BTC    ETH    BTC    ETH
Date                                                 
2022-01-01 00:00:00+00:00  False  False  False  False
2022-01-02 00:00:00+00:00  False  False  False  False
2022-01-03 00:00:00+00:00  False  False  False  False
2022-01-04 00:00:00+00:00  False  False  False  False
2022-01-05 00:00:00+00:00  False  False  False  False
...                          ...    ...    ...    ...
2024-12-27 00:00:00+00:00  False  False  False  False
2024-12-28 00:00:00+00:00  False  False  False  False
2024-12-29 00:00:00+00:00  False  False  False  False
2024-12-30 00:00:00+00:00  False  False  False  False
2024-12-31 00:00:00+00:00  False  False   True  False

[1096 rows x 4 columns]

In [18]:
pf = vbt.Portfolio.from_signals(comb_price, entries, exits)
pf.total_return()

fast_window  slow_window  symbol
10           30           BTC       0.579169
                          ETH       0.023467
20           30           BTC       0.579747
                          ETH       0.447563
Name: total_return, dtype: float64

In [19]:
mean_return = pf.total_return().groupby('symbol').mean()
mean_return.vbt.barplot(xaxis_title='Symbol', yaxis_title='Mean total return')

FigureWidget({
    'data': [{'name': 'total_return',
              'showlegend': True,
              'type': 'bar',
              'uid': 'bf34c2c2-cb10-4831-88be-47b02e33ba5a',
              'x': array(['BTC', 'ETH'], dtype=object),
              'y': array([0.57945785, 0.23551514])}],
    'layout': {'height': 350,
               'legend': {'orientation': 'h',
                          'traceorder': 'normal',
                          'x': 1,
                          'xanchor': 'right',
                          'y': 1.02,
                          'yanchor': 'bottom'},
               'margin': {'b': 30, 'l': 30, 'r': 30, 't': 30},
               'template': '...',
               'width': 700,
               'xaxis': {'title': {'text': 'Symbol'}},
               'yaxis': {'title': {'text': 'Mean total return'}}}
})